In [1]:
import pyarrow as pa
from pyarrow import parquet
import polars as pl
from pathlib import Path


In [2]:
import pyarrow.compute as pc

In [3]:
!pip install --upgrade pyarrow

In [4]:
paths = [*Path("parquet").glob("*.parquet")]
path = paths[10]


In [7]:
def read(path):
    dest = Path("simple_parquet") / path.name
    if dest.exists():
        return
    print(dest)
    m = parquet.read_table(path)

    name, occ, place = m['corrected_entry'].flatten()

    occ_name, occ_certainty = pc.list_flatten(occ).flatten()
    parents = pc.take(m['entry_uuid'], pc.list_parent_indices(occ))
    occs = pl.DataFrame({
        'occupation': pl.from_arrow(occ_name), 
        'entry_uuid': pl.from_arrow(parents), 
        'occ_certainty': pl.from_arrow(occ_certainty)
    })

    df = pl.from_arrow(m.select(['entry_uuid', 'directory_year', 'labeled_black', 'labeled_widow', 'complete_entry'])).lazy()

    name_array = name.combine_chunks()
    parents = pc.take(m['entry_uuid'], pc.list_parent_indices(name_array))
    #occs = pl.DataFrame({'occupation': pl.from_arrow(occ_name.combine_chunks()), 'entry_uuid': pl.from_arrow(parents), 'occ_certainty': pl.from_arrow(occ_certainty.combine_chunks())})
    names = pl.DataFrame({'entry_uuid': pl.from_arrow(parents), 'name': pl.from_arrow(name_array.flatten())})

    addr_name, addr_certainty, addr_type = pc.list_flatten(place).flatten()
    parents = pc.take(m['entry_uuid'], pc.list_parent_indices(place))
    #occs = pl.DataFrame({'occupation': pl.from_arrow(occ_name.combine_chunks()), 'entry_uuid': pl.from_arrow(parents), 'occ_certainty': pl.from_arrow(occ_certainty.combine_chunks())})
    first_type = pc.list_element(addr_type, 0)
    addr_type = pl.from_arrow(addr_type)
    places = pl.DataFrame(
      {
        'entry_uuid': pl.from_arrow(parents),
        'addr_name': pl.from_arrow(addr_name),
        'addr_certainty': pl.from_arrow(addr_certainty).cast(pl.Int8),
        'addr_h': addr_type.arr.contains("h"),
        'addr_r': addr_type.arr.contains("r"),
        'addr_ft': addr_type.arr.contains("ft")
      })

    combined = places.lazy().join(occs.lazy(), on = ['entry_uuid'], how = 'outer').join(names.lazy(), how='outer', on='entry_uuid').join(df, on=['entry_uuid'], how = 'outer')
    combined.head(10).collect()

    combined.select([
        pl.col('entry_uuid'),
        pl.col('addr_name'),
        "addr_certainty",
        'addr_h',
        'addr_r',
        'addr_ft',
        'occupation',
        pl.col('occ_certainty').cast(pl.Int8),#.alias('occ_certainty'),
        'name',
        'directory_year', 
        pl.col('labeled_black').cast(pl.Int8).cast(pl.Boolean),
        pl.col('labeled_widow').cast(pl.Int8).cast(pl.Boolean),
    ]).collect().to_parquet(dest)

In [8]:
for path in paths:
    read(path)

simple_parquet/1869.parquet


In [ ]:
len(pc.list_parent_indices(occ)

In [ ]:
len(occ)